## On edit Functions(.gs)

VERSION: `Update`

- Since every Day 3 session requires you to log 7 new entries for each of the 7 session folder that will be created (4 Encode/ 3 Retrive), it can get repetitive. The [Sync functions](D3Sync.ipynb) will help create the new rows and generate templates for most information. Meanwhile, Demo info like [Gender](Gender.ipynb) and [Age](Age_Calculator.ipynb) can be filled from the tab (given that it is in the database). The script below will help with other aspects of tracking. 

- There are a few functionalities defined here:
    - Tracking Date and Time
        - If you would to enter the current time, instead of writing it manually, simply type "now" in any cell and it will update itself to display the current date and time in the same format as in other cells. 
        - Similarly, typing "today" will yeild today's date in the same format
---      
- Highlighting same session in Day 3 sheets
    - When you are editing a file belonging to a participant, the script will automatically highlight (in red) files created in the same session (same study, same participant, same timepoint). This means editing one block will temporarily highlight all other blocks that belongs to that session. This could help you navigate the sheet better.
        
---
- Syncing  Cells
     - Certain cells are assumed to contain the same information as long as they belong to the same participant during the same session. When you make an edit in these cells, it will sync the other cells with whatever you added or removed. 
        - Those cells synced are defined in this line:  var columnsToSync = ["Date", "Transferred", "Sent by", "Device", "Method", "Collected", "Notes"];
        
        - *Main use is with Notes so that any errors is noted in all blocks*
        

In [ ]:
function onEdit(e) {
  var sheet = e.range.getSheet();
  if (sheet.getName() !== "D3E" && sheet.getName() !== "D3R") {
    return;
  }

  var range = e.range;
  var editedRow = range.getRow();
  var editedColumn = range.getColumn();
  var lastColumn = sheet.getLastColumn();
  var headers = sheet.getRange(1, 1, 1, lastColumn).getValues()[0];
  var participantIDCol = headers.indexOf("Participant_ID") + 1;
  var dateCol = headers.indexOf("Date") + 1;
  var newValue = e.value;
  var oldValue = e.oldValue;
  var columnsToSync = ["Date", "Transferred", "Sent by", "Device", "Method", "Collected", "Notes"];
  var indexesToSync = columnsToSync.map(function(header) {
    return headers.indexOf(header) + 1;
  }).filter(function(index) {
    return index > 0;
  });

  if (newValue && newValue.toLowerCase() === "now") {
    range.setValue(Utilities.formatDate(new Date(), Session.getScriptTimeZone(), "M/d/yy (HH:mm)"));
    return;
  }
  if (newValue && newValue.toLowerCase() === "today") {
    range.setValue(Utilities.formatDate(new Date(), Session.getScriptTimeZone(), "MMMM d, yyyy"));
    return;
  }

  var participantID = sheet.getRange(editedRow, participantIDCol).getValue();
  var sessionDate = sheet.getRange(editedRow, dateCol).getValue();

  highlightMatchingRows(sheet, participantID, sessionDate, editedRow, lastColumn);

  if (indexesToSync.includes(editedColumn)) {
    var dataRange = sheet.getDataRange();
    var data = dataRange.getValues();
    var matchingRowsCount = 0;
    var cellsUpdated = [];
    for (var i = 1; i < data.length; i++) {
      if (data[i][participantIDCol - 1] === participantID && 
          new Date(data[i][dateCol - 1]).getTime() === new Date(sessionDate).getTime()) {
        if ((i + 1) !== editedRow) {
          matchingRowsCount++;
          var cellToChange = sheet.getRange(i + 1, editedColumn);
          if (newValue === undefined) {
            if (oldValue !== undefined) {
              cellToChange.clearContent();
            }
          } else {
            cellToChange.setValue(newValue);
          }
          cellsUpdated.push(cellToChange.getA1Notation());
        }
      }
    }
    if (matchingRowsCount > 0) {
      var message = "Now editing: " + matchingRowsCount + " other cell(s) in the same session. Cells: " + cellsUpdated.join(", ");
      SpreadsheetApp.getActiveSpreadsheet().toast(message, 'Update Info', 10);
    }
  }
}

function highlightMatchingRows(sheet, participantID, sessionDate, editedRow, lastColumn) {
  var dataRange = sheet.getDataRange();
  var data = dataRange.getValues();
  var participantIDCol = sheet.getRange(1, 1, 1, lastColumn).getValues()[0].indexOf("Participant_ID") + 1;
  var dateCol = sheet.getRange(1, 1, 1, lastColumn).getValues()[0].indexOf("Date") + 1;
  var rowsToHighlight = [];
  for (var i = 1; i < data.length; i++) {
    if (data[i][participantIDCol - 1] === participantID && 
        new Date(data[i][dateCol - 1]).getTime() === new Date(sessionDate).getTime()) {
      rowsToHighlight.push(i + 1);
      sheet.getRange(i + 1, 1, 1, lastColumn).setBackground("#FFCCCC");
    }
  }
  SpreadsheetApp.flush();
  Utilities.sleep(10000);
  rowsToHighlight.forEach(function(row) {
    sheet.getRange(row, 1, 1, lastColumn).setBackground(null);
  });
}
